In [1]:
from math import ceil
import numpy as np
import pandas as pd
from pandas.plotting import scatter_matrix

from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.utils import resample

from basis_expansions.basis_expansions import NaturalCubicSpline
from regression_tools.dftransformers import (
    ColumnSelector, Identity,
    FeatureUnion, MapFeature,
    StandardScaler)
from regression_tools.plotting_tools import (plot_univariate_smooth,
                                bootstrap_train,
                                display_coef,
                                plot_bootstrap_coefs,
                                plot_partial_depenence,
                                plot_partial_dependences,
                                predicteds_vs_actuals)

%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('ggplot')

/Users/matthewdrury/anaconda3/lib/python3.6/site-packages/matplotlib/__init__.py:800: MatplotlibDeprecationWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  mplDeprecation)


In [2]:
balance = pd.read_csv('balance.csv')
balance = balance.drop(balance.columns[[0]], axis=1)

In [3]:
numeric_predictors = [
    "Income", "Limit", "Rating", "Cards", "Age", "Education"]

In [4]:
def simple_spline_specification(name, knots):
    select_name = "{}_select".format(name)
    spline_name = "{}_spline".format(name)
    return Pipeline([
        (select_name, ColumnSelector(name=name)),
        (spline_name, NaturalCubicSpline(knots=knots))
    ])

In [5]:
income_spec = simple_spline_specification("Income", [25, 50, 75, 100, 125])
limit_spec = simple_spline_specification("Limit", [3000, 4000, 5000, 6000, 7000, 8000])
rating_spec = simple_spline_specification("Rating", [200, 300, 400, 500, 600])
age_spec = simple_spline_specification("Age", [30, 40, 50, 60, 70])
education_spec = simple_spline_specification("Education", [8, 10, 12, 14, 16])

In [6]:
def simple_indicator_specification(var_name, levels):
    select_name = "{}_select".format(var_name)
    map_features = []
    for level in levels:
        indicator_name = "{}_{}_indicator".format(var_name, level)
        map_features.append(
            (indicator_name, MapFeature(lambda var: var == level, indicator_name))
        )
    return Pipeline([
        (select_name, ColumnSelector(name=var_name)),
        ("indicator_features", FeatureUnion(map_features))
    ])

In [7]:
cards_spec = simple_indicator_specification("Cards", [0, 1, 2, 3])
is_female = simple_indicator_specification("Gender", "Female")
student_spec = simple_indicator_specification("Student", "Yes")
married_spec = simple_indicator_specification("Married", "Yes")

In [8]:
continuous_features_scaled = Pipeline([
    ('continuous_features', FeatureUnion([
        ('income_fit', income_spec),
        ('limit_fit', limit_spec),
        ('rating_fit', rating_spec),
        ('age_fit', age_spec),
        ('education_fit', education_spec)])),
    ('standardizer', StandardScaler())
])
    
    
indicator_features = FeatureUnion([
    ('cards_fit', cards_spec),
    ('education_fit', education_spec),
    ('student_fit', student_spec),
    ('married_fit', married_spec)
])

balance_pipeline = FeatureUnion([
    ('continuous_features', continuous_features_scaled),
    ('indicator_features', indicator_features)
])

In [9]:
balance_pipeline.fit(balance)
balance_modeling = balance_pipeline.transform(balance)